<a href="https://colab.research.google.com/github/Ramees918/RJ/blob/main/Proj%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow matplotlib scikit-learn


# Task
Build an image classifier as a complete learning guide for an internship project.

## Data collection and preparation

### Subtask:
Gather a dataset of images suitable for the classification task. This may involve downloading a pre-existing dataset or collecting your own. Organize the data into categories and split it into training, validation, and test sets.


**Reasoning**:
The first step is to choose and download a suitable dataset. CIFAR-10 is a commonly used dataset for image classification and is readily available in libraries like TensorFlow. I will download and load the CIFAR-10 dataset.



In [ ]:
import tensorflow as tf
import numpy as np

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

# Combine train and test for easier splitting later
all_images = np.concatenate((train_images, test_images), axis=0)
all_labels = np.concatenate((train_labels, test_labels), axis=0)

print(f"Total number of images: {len(all_images)}")
print(f"Shape of images: {all_images.shape}")
print(f"Shape of labels: {all_labels.shape}")

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Total number of images: 60000
Shape of images: (60000, 32, 32, 3)
Shape of labels: (60000, 1)


**Reasoning**:
The next step is to split the combined dataset into training, validation, and test sets with a 70/15/15 split while ensuring the class distribution is balanced. I will use scikit-learn's `train_test_split` function for this.



In [ ]:
from sklearn.model_selection import train_test_split

# Split into training and temporary set (validation + test)
train_images, temp_images, train_labels, temp_labels = train_test_split(
    all_images, all_labels, test_size=0.3, random_state=42, stratify=all_labels
)

# Split the temporary set into validation and test sets
val_images, test_images, val_labels, test_labels = train_test_split(
    temp_images, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

print(f"Training set size: {len(train_images)}")
print(f"Validation set size: {len(val_images)}")
print(f"Test set size: {len(test_images)}")

Training set size: 42000
Validation set size: 9000
Test set size: 9000


## Data preprocessing

### Subtask:
Prepare the image data for use with a neural network. This typically includes resizing images to a consistent size, normalizing pixel values, and augmenting the data to increase the size and variability of the training set.


**Reasoning**:
Normalize the pixel values, convert labels to categorical format, define and apply data augmentation, create TensorFlow Datasets, and apply batching and prefetching.



In [ ]:
# Normalize pixel values
train_images = train_images / 255.0
val_images = val_images / 255.0
test_images = test_images / 255.0

# Convert labels to categorical format
num_classes = 10
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes)
val_labels = tf.keras.utils.to_categorical(val_labels, num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes)

# Define data augmentation pipeline
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

# Create TensorFlow Dataset objects
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
val_ds = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

# Apply data augmentation to the training dataset and batching/prefetching to all datasets
batch_size = 32
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE).batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)

print("Data preparation complete.")

Data preparation complete.


## Model selection and building

### Subtask:
Choose a suitable neural network architecture for image classification. This could be a pre-trained model like VGG, ResNet, or MobileNet, or a custom-built convolutional neural network (CNN). Build the model using a deep learning framework like TensorFlow or PyTorch.


**Reasoning**:
Build a simple convolutional neural network model using Keras Sequential API based on the subtask instructions.



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        65,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,570 (478.79 KB)

 Trainable params: 122,570 (478.79 KB)

 Non-trainable params: 0 (0.00 B)

## Model training

### Subtask:
Train the selected model on the prepared training data. This involves feeding the data through the network, calculating the loss, and updating the model's weights using an optimization algorithm. Monitor the training process using the validation set to prevent overfitting.


**Reasoning**:
Train the compiled model using the fit method, providing the training and validation datasets and setting the number of epochs.



In [ ]:
history = model.fit(
    train_ds,
    epochs=15,
    validation_data=val_ds
)

Epoch 1/15
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 75s 53ms/step - accuracy: 0.3207 - loss: 1.8499 - val_accuracy: 0.4781 - val_loss: 1.4425
Epoch 2/15
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 81s 52ms/step - accuracy: 0.4881 - loss: 1.4225 - val_accuracy: 0.5574 - val_loss: 1.2388
Epoch 3/15
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 81s 51ms/step - accuracy: 0.5385 - loss: 1.2872 - val_accuracy: 0.5851 - val_loss: 1.1683
Epoch 4/15
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 69s 53ms/step - accuracy: 0.5652 - loss: 1.2176 - val_accuracy: 0.5766 - val_loss: 1.2012
Epoch 5/15
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 69s 52ms/step - accuracy: 0.5907 - loss: 1.1564 - val_accuracy: 0.6068 - val_loss: 1.1257
Epoch 6/15
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 82s 52ms/step - accuracy: 0.6040 - loss: 1.1091 - val_accuracy: 0.6270 - val_loss: 1.0878
Epoch 7/15
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 71s 54ms/step - accuracy: 0.6205 - loss: 1.0721 - val_accuracy: 0.6238 - val_loss: 1.0960
Epoch 8/15
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 82s 54ms/step - accuracy: 0.6319 -

In [8]:
# 1. Introduction
# 2. Dataset Overview
# 3. CNN Model Architecture
# 4. Training and Results
# 5. Evaluation Metrics
